In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import itertools

import string
import re
import collections
from sklearn import  preprocessing
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.metrics import make_scorer, f1_score, accuracy_score, mean_absolute_error, confusion_matrix
import optuna


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV
from collections import Counter

np.set_printoptions(precision=4)

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn.metrics import accuracy_score
from matplotlib.colors import ListedColormap

import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
sns.set_style("whitegrid")
import warnings




In [3]:
simple = pd.read_csv("../input/two-sigma-connect-rental-listing-inquiries/sample_submission.csv.zip")
simple

In [4]:
train = pd.read_json('../input/two-sigma-connect-rental-listing-inquiries/train.json.zip')
test = pd.read_json('../input/two-sigma-connect-rental-listing-inquiries/test.json.zip')

# Basic Insight of Dataset (Базовое представление о наборе данных)


In [5]:
train. head(1)

In [6]:
train.shape

In [7]:
test.shape


In [8]:
print(train.columns.values)

In [9]:
print(test.columns.values)

In [10]:
train.describe()

In [11]:
train.describe(include = "all")

In [12]:
train.info()

In [13]:
test.info()

In [14]:
train.isnull().sum()

In [15]:
test.isnull().sum()

# Data visualization , pre-processing  and Feature engineering

### 'interest_level'


In [16]:
train.groupby(train['interest_level']).mean()

In [17]:
train['interest_level'].value_counts()

In [18]:
sns.set_style('whitegrid')
sns.countplot(x='interest_level',data=train,palette='RdBu_r')

### 'bathrooms'

In [19]:
# Value Counts
train['bathrooms'].value_counts()



In [20]:
train.groupby(['bathrooms']) ['interest_level'].value_counts(normalize=True)

In [21]:
train.pivot_table('price', 'bathrooms', 'interest_level', 'count').plot(kind='bar', stacked=True)

### 'bedrooms'

In [22]:
# Value Counts
train['bedrooms'].value_counts()

In [23]:
train.groupby(['bedrooms']) ['interest_level'].value_counts(normalize=True)

In [24]:
train.pivot_table('price', 'bedrooms', 'interest_level', 'count').plot(kind='bar', stacked=True)

In [25]:
# 'bathrooms' and 'bedrooms'
fig, axes = plt.subplots(ncols=2)
train.pivot_table('price', ['bathrooms'], 'interest_level', 'count').plot(ax=axes[0], title='bathrooms')
train.pivot_table('price', ['bedrooms'], 'interest_level', 'count').plot(ax=axes[1], title='bedrooms')

### Feature engineering

In [26]:
train["room_sum"] = train["bedrooms"]+train["bathrooms"]
train["bed_bath_dif"] = train["bedrooms"] - train["bathrooms"]
train.head(1)

In [27]:
test["room_sum"] = test["bedrooms"]+test["bathrooms"]
test["bed_bath_dif"] = test["bedrooms"] - test["bathrooms"]
test.head(1)

### 'building_id

In [28]:
# Most advertised buildings
train.building_id.value_counts().nlargest(10)

In [29]:
building_id = train["building_id"].value_counts()
train["building_count"] = train["building_id"].apply(lambda x:building_id[x])

train.head(1)

In [30]:
building_id = test["building_id"].value_counts()
test["building_count"] = test["building_id"].apply(lambda x:building_id[x])
test.head(1)

### 'created'

In [31]:
# Convertion to Python Date  (Преобразование в дату Python)
train.created = pd.to_datetime(train.created, format='%Y-%m-%d %H:%M:%S')
test.created = pd.to_datetime(test.created, format='%Y-%m-%d %H:%M:%S')

In [32]:
# Year, Month, Day of Week and Hour Features  (Год, Месяц, день недели и часы)
train["year"] = train.created.dt.year
train['month'] = train.created.dt.month
train["day"] = train.created.dt.day
train['day_of_week'] = train.created.dt.weekday
train['hour'] = train.created.dt.hour

train.head(1)

In [33]:
# Year, Month, Day of Week and Hour Features  (Год, Месяц, день недели и часы)
test["year"] = test.created.dt.year
test['month'] = test.created.dt.month
test["day"] = test.created.dt.day
test['day_of_week'] = test.created.dt.weekday
test['hour'] = test.created.dt.hour

test.head(1)

In [34]:
# Iterest per month
fig = plt.figure(figsize=(12,6))
ax = sns.countplot(x="month", hue="interest_level", hue_order=['low', 'medium', 'high'],
                   data=train);
plt.xlabel('Month');
plt.ylabel('Number of occurrences')

# Adding percents over bars
height = [p.get_height() for p in ax.patches]
ncol = int(len(height)/3)
total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
for i, p in enumerate(ax.patches):    
    ax.text(p.get_x()+p.get_width()/2,
            height[i] + 50,
            '{:1.0%}'.format(height[i]/total[i]),
            ha="center")


In [35]:
# Iterest per Day of Week
fig = plt.figure(figsize=(12,6))
ax = sns.countplot(x="day_of_week", hue="interest_level",
                   hue_order=['low', 'medium', 'high'], data=train);
plt.xlabel('Day of Week');
plt.ylabel('Number of occurrences');

### Adding percents over bars
height = [p.get_height() for p in ax.patches]
ncol = int(len(height)/3)
total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
for i, p in enumerate(ax.patches):    
    ax.text(p.get_x()+p.get_width()/2,
            height[i] + 50,
            '{:1.0%}'.format(height[i]/total[i]),
            ha="center")

In [36]:
# Iterest per Hour
fig = plt.figure(figsize=(12,6))
sns.countplot(x="hour", hue="interest_level", hue_order=['low', 'medium', 'high'], data=train);
plt.xlabel('Hour');
plt.ylabel('Number of occurrences');

### 'display_address'

In [37]:
# Number of unique Display Addresses
print('Number of Unique Display Addresses is {}'.format(train.display_address.value_counts().shape[0]))

In [38]:
# 20 most popular Display Addresses
train.display_address.value_counts().nlargest(20)

In [39]:
# Rent interest graph of New-York
sns.lmplot(x="longitude", y="latitude", fit_reg=False, hue='interest_level',
           hue_order=['low', 'medium', 'high'], size=9, scatter_kws={'alpha':0.4,'s':30},
           data=train[(train.longitude>train.longitude.quantile(0.1))
                        &(train.longitude<train.longitude.quantile(0.9))
                        &(train.latitude>train.latitude.quantile(0.1))                           
                        &(train.latitude<train.latitude.quantile(0.9))]);
plt.xlabel('Longitude');
plt.ylabel('Latitude');

### 'manager_id’


In [40]:
train.manager_id.value_counts().nlargest(10)

In [41]:
manager_id = train["manager_id"].value_counts()
train["manager_count"] = train["manager_id"].apply(lambda x:manager_id[x]) 
train.head(1)

In [42]:
manager_id = test["manager_id"].value_counts()
test["manager_count"] = test["manager_id"].apply(lambda x:manager_id[x]) 
test.head(1)

In [43]:
# Let's get a list of top 10 managers
top10managers = train.manager_id.value_counts().nlargest(10).index.tolist()
# ...and plot number of different Interest Level rental adverts for each of them
fig = plt.figure(figsize=(12,6))
ax = sns.countplot(x="manager_id", hue="interest_level",
data=train[train.manager_id.isin(top10managers)]);
plt.xlabel('Manager');
plt.ylabel('Number of advert occurrences');
### Manager_ids are too long. Let's remove them
plt.tick_params(
axis='x', # changes apply to the x-axis
which='both', # both major and minor ticks are affected
bottom='off', # ticks along the bottom edge are off
top='off', # ticks along the top edge are off
labelbottom='off');
plt.xticks([])
# Adding percents over bars
height = [0 if np.isnan(p.get_height()) else p.get_height() for p in ax.patches]
ncol = int(len(height)/3)
total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
for i, p in enumerate(ax.patches):
    ax.text(p.get_x()+p.get_width()/2,
            height[i] + 20,
            '{:1.0%}'.format(height[i]/total[i]),
            ha="center")


### 'photos'

In [44]:
# count of photos 
train["num_photos"] = train["photos"].apply(len)

train.head(1)

In [45]:
# count of photos 
test["num_photos"] = test["photos"].apply(len)
test.head(1)

### 

### price

In [46]:
# Value Counts
train['price'].value_counts()

In [47]:
# Price exploration
prices=train.groupby('interest_level', as_index=False)['price'].mean()
colors = ['lightcoral','gold','lightblue']

fig=plt.figure(figsize=(8,6))
plt.bar(prices.interest_level, prices.price, color=colors, width=0.5, alpha=0.8)
#set titles
plt.xlabel('Interest level')
plt.ylabel('Average price')
plt.title('Average price across interest level')
plt.show()

In [48]:
# Price exploration
fig = plt.figure(figsize=(12,12))
# Price distribution
sns.distplot(train.price[train.price<=train.price.quantile(0.99)], ax=plt.subplot(211));
plt.xlabel('Price');
plt.ylabel('Density');

# Average Price per Interest Level
sns.barplot(x="interest_level", y="price", order=['low', 'medium', 'high'],
            data=train, ax=plt.subplot(223));
plt.xlabel('Interest Level');
plt.ylabel('Price');

# Violinplot of price for every Interest Level
sns.violinplot(x="interest_level", y="price", order=['low', 'medium', 'high'],
               data=train[train.price<=train.price.quantile(0.99)],
               ax=plt.subplot(224));
plt.xlabel('Interest Level');
plt.ylabel('Price');

### 'description’

In [49]:
train['description'].iloc[0]

In [50]:
# REMOVE UNNECESSARY WORDS FROM DESCRIPTION (УДАЛИТЬ НЕНУЖНЫЕ СЛОВА ИЗ ОПИСАНИЯ)
train['description'] = train['description'].apply(lambda x: x.replace("<br />", ""))
train['description'] = train['description'].apply(lambda x: x.replace("br", ""))
train['description'] = train['description'].apply(lambda x: x.replace("<p><a", ""))
train['description'].iloc[0]


In [51]:
# REMOVE UNNECESSARY WORDS FROM DESCRIPTION (УДАЛИТЬ НЕНУЖНЫЕ СЛОВА ИЗ ОПИСАНИЯ)
test['description'] = test['description'].apply(lambda x: x.replace("<br />", ""))
test['description'] = test['description'].apply(lambda x: x.replace("br", ""))
test['description'] = test['description'].apply(lambda x: x.replace("<p><a", ""))
test['description'].iloc[0]

### 'features'

In [52]:
# count of "features" 
train["num_features"] = train["features"].apply(len)
train.head(1)

In [53]:
# count of "features" 
test["num_features"] = test["features"].apply(len)
test.head(1)


In [54]:
# CONVERT LOWER ALL OF WORDS
train[["features"]] = train[["features"]].apply(
    lambda _: [list(map(str.strip, map(str.lower, x))) for x in _])
train[["features"]]


In [55]:
# CONVERT LOWER ALL OF WORDS
test[["features"]] = test[["features"]].apply(
    lambda _: [list(map(str.strip, map(str.lower, x))) for x in _])
test[["features"]]


### 'features'  MOST FREQUENT FEATURES EXTRACTION  (ИЗВЛЕЧЕНИЕ НАИБОЛЕЕ ЧАСТОТЫХ ФУНКЦИЙ)

In [56]:
feature_value_train = train['features'].tolist()
feature_value_test = test['features'].tolist()


In [57]:
feature_lst_train = []
for i in range(len(feature_value_train)):
    feature_lst_train += feature_value_train[i]
    
uniq_feature_train = list(set(feature_lst_train))

# print(uniq_feature) #all unique features
len(uniq_feature_train)
    
    

In [58]:
feature_lst_test = []
for i in range(len(feature_value_test)):
    feature_lst_test += feature_value_test[i]
    
uniq_feature_test = list(set(feature_lst_test))

# print(uniq_feature) #all unique features
len(uniq_feature_test)

In [59]:
# see the frequency of each feature
import collections
def most_common(lst):
    features = collections.Counter(lst)
    feature_value = features.keys()
    frequency = features.values()
    data = [('feature_value', feature_value),
            ('frequency', frequency),]    
    df = pd.DataFrame.from_dict(dict(data))
    return df.sort_values(by = 'frequency', ascending = False)

df_features_train = most_common(feature_lst_train)

df_features_test = most_common(feature_lst_test)

df_features_train


In [60]:
df_features_train.head(20)


In [61]:
df_features_test.head(20)

In [62]:
def newColumn(name, df, series):
    feature = pd.Series(0,df.index,name = name)# data : 0
    for row,word in enumerate(series):
        if name in word:
            feature.iloc[row] = 1
    df[name] = feature # feature : series ; value in series : 1 or 0
    return df

# select features based on frequency
facilities = [
'elevator', 'cats allowed', 'hardwood floors', 'dogs allowed', 'doorman', 'dishwasher', 'no fee', 'laundry in building', 'fitness center',
'laundry in unit', 'pre-war', 'roof deck', 'outdoor space','dining room', 'high speed internet', 'balcony', 'swimming pool',
'new construction', 'terrace']
for name in facilities:
    train = newColumn(name, train, train['features'])
    test = newColumn(name, test, test['features'])
train.head(1)

In [63]:
test.head(1)

In [65]:
#WORDCLOUD FOR DESCRIPTION AND DISPLAY ADDRESS
#Preprocessing
text = ''
text_da = ''


for ind, row in train.iterrows():
    for feature in row['features']:
        text = " ".join([text, "_".join(feature.strip().split(" "))])
    text_da = " ".join([text_da,"_".join(row['display_address'].strip().split(" "))])
    
text = text.strip()
text_da = text_da.strip()



# wordcloud for features
plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='white', width=600, height=300, max_font_size=50, max_words=40).generate(text)
wordcloud.recolor(random_state=0)
plt.imshow(wordcloud)
plt.title("Wordcloud for features", fontsize=30)
plt.axis("off")
plt.show()




# wordcloud for display address
plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='gold', width=600, height=300, max_font_size=50, max_words=40).generate(text_da)
wordcloud.recolor(random_state=0)
plt.imshow(wordcloud)
plt.title("Wordcloud for Display Address", fontsize=30)
plt.axis("off")
plt.show()






# Feature Selection  (Выбор функции)

In [66]:
train.columns

In [67]:
train.isnull().sum()

In [68]:
test.isnull().sum()

In [69]:
# LABEL ECONDING FOR CATEGORICAL VARIABLES
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train[f].dtype=='object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train[f].values) + list(test[f].values))
            train[f] = lbl.transform(list(train[f].values))
            test[f] = lbl.transform(list(test[f].values))

In [70]:
X = train[['bathrooms', 'bedrooms', 'latitude',  'longitude', 'price', 
       'room_sum', 'bed_bath_dif','display_address','manager_id', 'building_id',
       'year', 'month', 'day', 'day_of_week', 'hour','street_address',
       'building_count', 'manager_count', 'num_photos', 
        'num_features', 'elevator', 'cats allowed', 'hardwood floors', 'dogs allowed',
       'doorman', 'dishwasher', 'no fee', 'laundry in building',
       'fitness center', 'laundry in unit', 'pre-war', 'roof deck',
       'outdoor space', 'dining room', 'high speed internet', 'balcony',
       'swimming pool', 'new construction', 'terrace']]


In [71]:
TestX = test[['bathrooms', 'bedrooms',  'latitude',  'longitude', 'price',  
       'room_sum', 'bed_bath_dif', 'display_address','manager_id','building_id',
       'year', 'month', 'day', 'day_of_week', 'hour','street_address',
       'building_count', 'manager_count', 'num_photos', 'num_features',
       'elevator', 'cats allowed', 'hardwood floors', 'dogs allowed',
       'doorman', 'dishwasher', 'no fee', 'laundry in building',
       'fitness center', 'laundry in unit', 'pre-war', 'roof deck',
       'outdoor space', 'dining room', 'high speed internet', 'balcony',
       'swimming pool', 'new construction', 'terrace']]

In [72]:
train['target'] = train['interest_level'].map({'high':0, 'medium':1, 'low':2})
train['target']

# Classification

### XGBOOST (алгоритм градиентного бустинга на деревьях)

In [73]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import make_scorer, f1_score, accuracy_score, mean_absolute_error, confusion_matrix

In [74]:

y = train.target

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.15,random_state = 42,stratify = y)


# Импорт XGBoost и создание необходимых объектов.
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)


In [75]:
kf = KFold(n_splits=5, shuffle=False)
X_train = X_train.values
y_train = y_train.values

scores = []

for train, test in kf.split(X_train, y_train):
    model = XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth = 10)
    model.fit(X_train[train], y_train[train])
    scores.append(model.score(X_train[test], y_train[test]))

In [76]:
def objective(trial):
    params = {
        'booster':trial.suggest_categorical('booster', ['gbtree', 'dart', 'gblinear']),
        'learning_rate':trial.suggest_loguniform("learning_rate", 0.01, 0.1),
        'max_depth':trial.suggest_int("max_depth", 3, 11),
        'subsample':trial.suggest_uniform("subsample", 0.0, 1.0),
        'colsample_bytree':trial.suggest_uniform("colsample_bytree", 0.0, 1.0),
    }

    model = XGBClassifier(**params)
    cv = KFold(n_splits=3, shuffle=True, random_state=None)
    scorer = make_scorer(f1_score, greater_is_better=True)
    
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    f1_scores = f1_score(y_test, pred_labels, average='micro')
    return f1_scores

In [77]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

In [78]:
new_params = study.best_params

new_model = XGBClassifier(**new_params)
new_model.fit(X, y)
preds = new_model.predict(X_test)

print('Optimized SuperLearner accuracy: ', accuracy_score(y_test, preds))
print('Optimized SuperLearner f1-score: ', f1_score(y_test, preds, average='micro'))

In [79]:
print("All of accuracies")
print(scores)
print("Mean of accuracies")
print(np.mean(scores))

In [80]:
y = new_model.predict_proba(TestX)

In [81]:
labels2idx = {label: i for i, label in enumerate(new_model.classes_)}
labels2idx

In [82]:

test = pd.read_json('../input/two-sigma-connect-rental-listing-inquiries/test.json.zip')



In [83]:


sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]


for label in [0, 1, 2]:
    sub[label] = y[:, labels2idx[label]]
    
sub.head(30)

In [84]:
sub.rename(columns={0:'high',1:'medium',2:'low'}, inplace = True)
sub

In [ ]:
sub.to_csv("submission.csv", index=False)
print("Your submission was successfully saved!")